<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>RUNNING FANAL ANALYSIS</p><br>

This notebook illustrates how to run the FANAL analysys departing from NEXUS files.

The result of the FANAL analysis will be the probability of the simulated events to be interpreted as a signal event so, although we typycally refer to it as "*rejection factor*", strictly speaking it is an "*acceptance factor*". It also provides in a Pandas DataFrame format all the information related to "events", "tracks" and "vovels" generated during the analysis.

The process to run the analysis consists in creating a FanalSetup instance with its details (detector, input nexus files, output fils, analysis parameters ...), and the action of running the analysis. This NB illustrates 2 different ways to build the setup and run the analysis:
1. Loading the Setup config parameters from a config file (json formatted).
2. Building the Setup by providing by hand all the parameters needed.

`FANAL analysis` process is run in a per-event basis and basically consists on a series of filters that events must fulfill to pass to the next analysis step. Events passing all the filters are considered as signal events. Following, a summary of the main steps of the analysis is presented:
> * `MC filter`: Checks that MC event energy fits into limits, there is no signal in BUFFER, and there event has a single S1 signal.
> * Reconstructed hits = MC ionization hits with energy and positions smeared; and shifted z_position (if needed by hit time).
> * `energy filter`: Checks if the smeared event energy fits into limits.
> * Voxelize reconstructed hits using Paolina (IC / v2) package.
> * `fiducial filter`: Checks if there is any voxel in the non-fiducial region with energy higher than threshold.
> * Make tracks from voxels using Paolina (IC / v2) package.
> * `track filter`: Checks if the number of tracks is lower or equal the number set by parameter. Currently set to 1.
> * Get blobs from tracks using Paolina (IC / v2) package.
> * `blob filter`: Checks if blob energies are higher than a certain energy threshold set by parameter.
> * `ROY filter`: Checks if the event smeared energy fts into ROI limits set by parameter.

The NB "fanal_results.ipynb" shows the different data stored in FANAL analysis with associated plots.

#### Setting general stuff

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

In [4]:
%load_ext memory_profiler
%load_ext line_profiler

import line_profiler
profile = line_profiler.LineProfiler()

#### Importing

In [5]:
# General importings

import glob
import json

In [6]:
# Specific IC stuff

import invisible_cities.core.system_of_units  as units

In [7]:
# Specific FANAL stuff

from fanal.fanal_setup       import Setup
from fanal.core.fanal_types  import BBAnalysisParams

# RUNNING FANAL

## from a given config file (json format) ...

In [8]:
config_fnames = glob.glob("../config/*.config")
print ('Available config files:\n')
for config_fname in config_fnames: print(config_fname) 

Available config files:

../config/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.config
../config/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.config
../config/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.config


In [9]:
#config_fname = "../config/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.config"
config_fname = "../config/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.config"

fanal_setup = Setup.from_config_file(config_fname)

In [12]:
%%time

results_df, events_df, tracks_df, voxels_df = fanal_setup.run_analysis()

*******************************************************************************
*** Detector:          NEXT100
*** Reconstructing:    bb0nu events
*** Input  files:      /Users/Javi/Development/FANAL/data/next100/bb0nu/sim/*.h5  (5 files)
*** Output file:       /Users/Javi/Development/FANAL/data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.h5
*** Buffer energy th.: 2.0 keV
*** Transverse   diff: 0.00  mm/cm**0.5
*** Longitudinal diff: 0.00  mm/cm**0.5
*** Energy Resolution: 0.70% fwhm at Qbb
*** Recons. procedure: paolina_ic
*** Voxel Size:        (10.0, 10.0, 10.0) mm  -  strict: False
*** Voxel energy th.:  2.0 keV
*** Track energy th.:  3.0 keV
*** Max num Tracks:    1
*** Blob radius:       18.0 mm
*** Blob energy th.:   350.0 keV
*** ROI energy limits: (2449.0, 2466.0) keV
*******************************************************************************


*** Processing /Users/Javi/Development/FANAL/data/next100/bb0nu/sim/next100.bb0nu.000.next.h5  (100 events) ...

## from parameters ...

In [14]:
# General stuff
det_name        = 'NEXT100'
event_type      = 'bb0nu'
input_fname     = '../data/next100/bb0nu/sim/*.h5'
output_fname    = '../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5'
verbosity_level = 'WARNING' # ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')

bb_analysis_params = \
    BBAnalysisParams(buffer_Eth        = 0.   * units.keV,
                     trans_diff        = 0.0  * units.mm / units.cm**0.5,
                     long_diff         = 0.0  * units.mm / units.cm**0.5,
                     fwhm              = 0.7  * units.perCent,
                     e_min             = 2.4  * units.MeV,
                     e_max             = 2.5  * units.MeV,
                     procedure         = "paolina_2",
                     voxel_size_x      = 10.  * units.mm,
                     voxel_size_y      = 10.  * units.mm,
                     voxel_size_z      = 10.  * units.mm,
                     strict_voxel_size = False,
                     barycenter        = True,
                     voxel_Eth         = 0.   * units.keV,
                     veto_width        = 20.  * units.mm,
                     veto_Eth          = 10.  * units.keV,
                     contiguity        = 15.  * units.mm,
                     track_Eth         = 0.   * units.keV,
                     max_num_tracks    = 1,
                     blob_radius       = 21.  * units.mm,
                     blob_Eth          = 350  * units.keV,
                     roi_Emin          = 2454 * units.keV,
                     roi_Emax          = 2471 * units.keV
                    )


# Executing FANAL
fanal_setup = Setup(det_name           = det_name,
                    event_type         = event_type,
                    input_fname        = input_fname,
                    output_fname       = output_fname,
                    bb_analysis_params = bb_analysis_params,
                    verbosity          = verbosity_level)

In [15]:
%%time

results_df, events_df, tracks_df, voxels_df = fanal_setup.run_analysis()

*******************************************************************************
*** Detector:          NEXT100
*** Reconstructing:    bb0nu events
*** Input  files:      ../data/next100/bb0nu/sim/*.h5  (5 files)
*** Output file:       ../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5
*** Buffer energy th.: 0.0 keV
*** Transverse   diff: 0.00  mm/cm**0.5
*** Longitudinal diff: 0.00  mm/cm**0.5
*** Energy Resolution: 0.70% fwhm at Qbb
*** Recons. procedure: paolina_2
*** Voxel Size:        (10.0, 10.0, 10.0) mm  -  barycenter: True
*** Voxel energy th.:  0.0 keV
*** Contiguity      :  15.0 mm
*** Track energy th.:  0.0 keV
*** Max num Tracks:    1
*** Blob radius:       21.0 mm
*** Blob energy th.:   350.0 keV
*** ROI energy limits: (2454.0, 2471.0) keV
*******************************************************************************


*** Processing ../data/next100/bb0nu/sim/next100.bb0nu.000.next.h5  (100 events) ...

* Num analyzed events: 1
* Num analyzed events: 2

# SHOWING FANAL RESULTS

In [16]:
fanal_setup

*******************************************************************************
*** Detector:          NEXT100
*** Reconstructing:    bb0nu events
*** Input  files:      ../data/next100/bb0nu/sim/*.h5  (5 files)
*** Output file:       ../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5
*** Buffer energy th.: 0.0 keV
*** Transverse   diff: 0.00  mm/cm**0.5
*** Longitudinal diff: 0.00  mm/cm**0.5
*** Energy Resolution: 0.70% fwhm at Qbb
*** Recons. procedure: paolina_2
*** Voxel Size:        (10.0, 10.0, 10.0) mm  -  barycenter: True
*** Voxel energy th.:  0.0 keV
*** Contiguity      :  15.0 mm
*** Track energy th.:  0.0 keV
*** Max num Tracks:    1
*** Blob radius:       21.0 mm
*** Blob energy th.:   350.0 keV
*** ROI energy limits: (2454.0, 2471.0) keV
*******************************************************************************

In [17]:
events_df.columns

Index(['num_mcParts', 'num_mcHits', 'mc_energy', 'mc_filter', 'sm_energy',
       'energy_filter', 'num_voxels', 'voxel_size_x', 'voxel_size_y',
       'voxel_size_z', 'veto_energy', 'fiduc_filter', 'num_tracks',
       'track_length', 'track_filter', 'blob1_energy', 'blob2_energy',
       'blob_filter', 'roi_filter'],
      dtype='object')

In [18]:
events_df[['num_mcParts', 'num_mcHits', 'mc_energy', 'sm_energy', 'energy_filter', 'num_voxels',
           'fiduc_filter', 'num_tracks', 'track_length', 'track_filter',
           'blob1_energy', 'blob2_energy', 'blob_filter', 'roi_filter']].head()

,num_mcParts,num_mcHits,mc_energy,sm_energy,energy_filter,num_voxels,fiduc_filter,num_tracks,track_length,track_filter,blob1_energy,blob2_energy,blob_filter,roi_filter
event_id,,,,,,,,,,,,,,
0,25,310,2.457830,2.456754,True,18,True,1,82.019402,True,0.887912,0.857141,True,True
1,22,310,2.457830,2.454281,True,31,True,1,107.467032,True,0.744729,0.664926,True,True
2,25,146,1.020527,NaN,False,-1,False,-1,NaN,False,NaN,NaN,False,False
3,22,330,2.328514,NaN,False,-1,False,-1,NaN,False,NaN,NaN,False,False
4,30,337,2.457830,2.460373,True,21,True,3,NaN,False,NaN,NaN,False,False


In [19]:
tracks_df.head()

energy      length  num_voxels    t_ext1_x    t_ext1_y  \
event_id track_id                                                             
0        0         2.426518   82.019402          18  316.199402 -311.569519   
1        0         2.407348  107.467032          31  128.015320  349.761017   
4        0         2.189262   -1.000000          19         NaN         NaN   
         1         0.146051   -1.000000           1         NaN         NaN   
         2         0.099771   -1.000000           1         NaN         NaN   

                     t_ext1_z    t_ext2_x    t_ext2_y    t_ext2_z     blob1_x  \
event_id track_id                                                               
0        0         221.003082  342.226501 -288.589630  153.732697  314.759534   
1        0         760.370239  181.914307  337.398102  757.257202  127.940644   
4        0                NaN         NaN         NaN         NaN         NaN   
         1                NaN         NaN         NaN         NaN         NaN   
         2                NaN         NaN         NaN         NaN         NaN   

                      blob1_y     blob1_z  blob1_energy  blob1_num_hits  \
event_id track_id                                                         
0        0        -311.537672  217.332581      0.887912               4   
1        0         349.683986  760.589985      0.744729               7   
4        0                NaN         NaN           NaN               0   
         1                NaN         NaN           NaN               0   
         2                NaN         NaN           NaN               0   

                      blob2_x     blob2_y     blob2_z  blob2_energy  \
event_id track_id                                                     
0        0         342.897182 -288.003118  155.999800      0.857141   
1        0         179.850730  336.573109  755.634106      0.664926   
4        0                NaN         NaN         NaN           NaN   
         1                NaN         NaN         NaN           NaN   
         2                NaN         NaN         NaN           NaN   

                   blob2_num_hits  ovlp_energy  
event_id track_id                               
0        0                      7          0.0  
1        0                      4          0.0  
4        0                      0          NaN  
         1                      0          NaN  
         2                      0          NaN

In [20]:
#tracks_df[tracks_df.ovlp_energy > 0.]

In [21]:
voxels_df.head()

x           y           z    energy
event_id track_id voxel_id                                              
0        0        0         316.318010 -310.624221  197.848884  0.100676
                  1         316.776795 -310.809279  206.536098  0.090680
                  2         314.759534 -311.537672  217.332581  0.604539
                  3         314.970880 -307.771823  213.355152  0.092017
                  4         324.274473 -311.325437  196.732273  0.076632

In [22]:
results_df

,events
simulated,500
stored,500
analyzed,500
mc_filter,357
energy_filter,357
fiduc_filter,311
track_filter,216
blob_filter,197
roi_filter,132


In [23]:
from fanal.containers.events import EventCounter

print(EventCounter(**results_df.events))

* Event counters ...
  Simulated    :        500  (1.00e+00)
  Stored       :        500  (1.00e+00)
  Analyzed     :        500  (1.00e+00)
  MC     filter:        357  (7.14e-01)
  Energy filter:        357  (7.14e-01)
  Fiduc. filter:        311  (6.22e-01)
  Track  filter:        216  (4.32e-01)
  Blob   filter:        197  (3.94e-01)
  ROI    filter:        132  (2.64e-01)

